In [ ]:
# The code was removed by Watson Studio for sharing.

## IBM Data Science Capstone Project


## Introduction
### Best Nieghborhood for technology Company in Toronto
With over $8 billion in contributions to annual employee wages, Toronto's technology sector is one of the fast-growing in north America. Its great infrastructure, the ease of connection to the rest of the world, good transit, and strong business-centric nature makes Toronto very attractive to international firms and companies. DexTR, a technology company based in San Francisco: engaged in design graphics processing units (GPUs), system chip for smartphone and vehicle navigation processors, is looking to open the second largest center in North America.

After months of regress research and analysis, DexTR has been decided Toronto as the next logical location for expansion in North America. Now that the decision has been made, the objective of this capstone is to analyze and select the optimal location for the DexTR office in the city of Toronto. While not limited to, the criteria below are some of the guiding measures.



#### Import libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # Library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup # library to parse HTML and XML documents

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries loaded.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

### Scrape Wikipedia page to obtain data and transform into panda datafram using BeatifulSoup

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #wikipedia link to the table
headers = {'User-Agent': 'Mozilla/5.0'} #setting my browser

In [5]:
response = requests.get(url, headers = headers)
soup = BeautifulSoup(response.content, "html.parser")

In [6]:
# creating empty list for the three in the table to store data
postCodeList = []
boroughList = []
neighborList = []

In [7]:
## accessing the table and saving it under vairable
wikitable = soup.find('table', class_="wikitable")

In [8]:
# iterating through the table to get the contents
for row in wikitable.find_all("tr"): #finding row in the table
    cell = row.find_all("td") #finding cells in the row
    if(len(cell) > 0):
        #appending list to corresponding column
        postCodeList.append(cell[0].text)
        boroughList.append(cell[1].text)
        neighborList.append(cell[2].text.rstrip('\n'))

### Transforming the data in to pandas DataFram

In [9]:
#creating columns for corrosponding lists
columns ={"PostCode": postCodeList, "Borough": boroughList, "Neighborhood": neighborList}
wiki_df = pd.DataFrame(columns)
wiki_df.head(12)

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Queen's Park


## Cleaning data: drop cells with borough not assigned

In [10]:
# drop cells with borough not assigned
wiki_df_dropna = wiki_df[wiki_df.Borough != "Not assigned"].reset_index(drop=True)
wiki_df_dropna.head(10)

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [11]:
# group neighborhoods with same postal code area
wiki_df_grouped = wiki_df_dropna.groupby(["PostCode", "Borough"],sort=False, as_index=False).agg(lambda x: ", ".join(x))
wiki_df.reset_index(inplace=True)
wiki_df_grouped.head(10)

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [12]:
# for a borough with no  neighborhood assigned, set borough as neighborhood 
for index, row in wiki_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
wiki_df_grouped.head(10).set_index("PostCode", inplace=True)

## Printing the shape of the datafram


In [17]:
wiki_df.shape
wiki_df_grouped.shape

(103, 3)

In [23]:
# The code was removed by Watson Studio for sharing.

In [24]:
# Load the coordinates from the csv file
coordinates = pd.read_csv(Geospatial_Coordinates)

# renaming postal code column
coordinates.rename(columns={"Postal Code": "PostCode"}, inplace=True)
coordinates.head()


,PostCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging the table and coordinates

In [26]:
#Merging the table and coordinates
merged_df = wiki_df_grouped.merge(coordinates, on="PostCode", how="left")
merged_df.head(12)

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Exploring and clustering the neighborhoods in Toronto using geopy to define a geocoder instance

In [27]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate for city of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate for city of Toronto are 43.653963, -79.387207.
